In [ ]:
!pip install mysql-connector-python
!pip install Faker
!pip install django

In [1]:
import random
from faker import Faker
import mysql.connector
from mysql.connector import Error
import os
from django.db import models
from django.contrib import admin
from django.urls import path, include

In [2]:
file_path = os.getcwd()

In [3]:
import random
from faker import Faker
fake = Faker()

relation_files = {
    'Customer': 'customer_records.txt',
    'Book': 'book_records.txt',
    'Author': 'author_records.txt',
    'Publisher': 'publisher_records.txt',
    'Order': 'order_records.txt',
    'BookAuthor': 'book_author_records.txt',
    'OrderBook': 'order_book_records.txt',
    'AuthorPublisher': 'author_publisher_records.txt'
}

num_records = 10000

# Generate Customer records
with open(relation_files['Customer'], 'w') as f:
    myEmailSet = set()
    for i in range(1, num_records + 1):
        email = fake.email()
        while email in myEmailSet:
            email = fake.email()
        myEmailSet.add(email)
        record = f"{i};{fake.first_name()};{fake.last_name()};{email};{fake.phone_number()};"
        record += fake.address().replace('\n', ', ') + "\n"
        f.write(record)

# Generate Book records
with open(relation_files['Book'], 'w') as f:
    myISBNSet = set()
    for i in range(1, num_records + 1):
        isbn = fake.isbn13()
        while isbn in myISBNSet:
            isbn = fake.isbn13()
        myISBNSet.add(isbn)
        record = f"{i};{fake.sentence()[:-1]};{isbn};{fake.date_between(start_date='-10y', end_date='today')};{random.uniform(1, 100):.2f};{random.randint(1, num_records)}\n"
        f.write(record)

# Generate Author records
with open(relation_files['Author'], 'w') as f:
    for i in range(1, num_records + 1):
        record = f"{i};{fake.first_name()};{fake.last_name()};{fake.date_of_birth(tzinfo=None, minimum_age=20, maximum_age=100)};"
        record += fake.text().replace('\n', ' ') + "\n"
        f.write(record)

# Generate Publisher records
with open(relation_files['Publisher'], 'w') as f:
    myEmailSet = set()
    for i in range(1, num_records + 1):
        email = fake.email()
        while email in myEmailSet:
            email = fake.email()
        myEmailSet.add(email)
        record = f"{i};{fake.company()};{email};{fake.phone_number()};"
        record += fake.address().replace('\n', ', ') + "\n"
        f.write(record)

# Generate Order records
with open(relation_files['Order'], 'w') as f:
    for i in range(1, num_records + 1):
        record = f"{i};{random.randint(1, num_records)};{fake.date_between(start_date='-5y', end_date='today')};{random.uniform(1, 1000):.2f};"
        record += fake.address().replace('\n', ', ') + "\n"
        f.write(record)

# Generate BookAuthor records
with open(relation_files['BookAuthor'], 'w') as f:
    book_ids = random.sample(range(1, num_records + 1),num_records)
    author_ids = random.sample(range(1, num_records + 1),num_records)
    for i in range(num_records):
        record = f"{book_ids[i]};{author_ids[i]}\n"
        f.write(record)

# Generate OrderBook records
with open(relation_files['OrderBook'], 'w') as f:
    order_ids = random.sample(range(1, num_records + 1),num_records)
    book_ids = random.sample(range(1, num_records + 1),num_records)
    for i in range(num_records):
        record = f"{order_ids[i]};{book_ids[i]};{random.randint(1, 10)}\n"
        f.write(record)

# Generate AuthorPublisher records
with open(relation_files['AuthorPublisher'], 'w') as f:
    author_ids = random.sample(range(1, num_records + 1),num_records)
    publisher_ids = random.sample(range(1, num_records + 1),num_records)
    for i in range(num_records):
        record = f"{author_ids[i]};{publisher_ids[i]}\n"
        f.write(record)

print("All record files have been created.")


All record files have been created.


In [4]:
import mysql.connector
from mysql.connector import Error

# MySQL connection settings
user = "root"
password = "YOUR MYSQL PASSWORD"
host = "localhost"
database_name = "ONLINEBOOKSTORE"

# Connect to the MySQL server
def create_connection(database=None):
    try:
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return connection
    except Error as e:
        print(f"The error '{e}' occurred")

# Execute SQL queries
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
# Create a new database
def create_database(connection, database_name):
    cursor = connection.cursor()
    try:
        cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
        cursor.execute(f"CREATE DATABASE {database_name}")
        connection.commit()
        print(f"Database '{database_name}' created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

# Insert data into tables
def insert_data(connection, table, columns, file):
    cursor = connection.cursor()
    with open(file, 'r') as f:
        for line in f:
            values = line.strip().split(';')
            query = f"INSERT INTO {table} ({columns}) VALUES ({', '.join(['%s'] * len(values))})"
            try:
                cursor.execute(query, values)
            except Exception as e:
                print(e)
                pass
    connection.commit()
    print(f"Data inserted into {table}")


In [5]:
# Establish connection without selecting a database
connection = create_connection()

# Create the database
create_database(connection, database_name)

# Close the initial connection
connection.close()

# Establish connection with the newly created database
connection = create_connection(database=database_name)

Database 'ONLINEBOOKSTORE' created successfully


In [6]:
# Create tables
create_table_queries = [
    '''
    CREATE TABLE IF NOT EXISTS Customer (
        customer_id INT AUTO_INCREMENT PRIMARY KEY,
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255) NOT NULL,
        email VARCHAR(255) UNIQUE NOT NULL,
        phone_number VARCHAR(255),
        address TEXT
    );
    ''',
    '''
    CREATE TABLE IF NOT EXISTS Book (
        book_id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    isbn VARCHAR(255) UNIQUE NOT NULL,
    publication_date DATE,
    price DECIMAL(10, 2) NOT NULL,
    publisher_id INT
);
''',
'''
CREATE TABLE IF NOT EXISTS Author (
    author_id INT AUTO_INCREMENT PRIMARY KEY,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    bio TEXT,
    birth_date DATE
);
''',
'''
CREATE TABLE IF NOT EXISTS Publisher (
    publisher_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    address TEXT,
    email VARCHAR(255) UNIQUE NOT NULL,
    phone_number VARCHAR(255)
);
''',
'''
CREATE TABLE IF NOT EXISTS `Order` (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT NOT NULL,
    order_date DATE,
    total_amount DECIMAL(10, 2) NOT NULL,
    shipping_address TEXT,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id)
);
''',
'''
CREATE TABLE IF NOT EXISTS BookAuthor (
    book_id INT,
    author_id INT,
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES Book(book_id),
    FOREIGN KEY (author_id) REFERENCES Author(author_id)
);
''',
'''
CREATE TABLE IF NOT EXISTS OrderBook (
    order_id INT,
    book_id INT,
    quantity INT NOT NULL,
    PRIMARY KEY (order_id, book_id),
    FOREIGN KEY (order_id) REFERENCES `Order`(order_id),
    FOREIGN KEY (book_id) REFERENCES Book(book_id)
);
''',
'''
CREATE TABLE IF NOT EXISTS AuthorPublisher (
    author_id INT,
    publisher_id INT,
    PRIMARY KEY (author_id, publisher_id),
    FOREIGN KEY (author_id) REFERENCES Author(author_id),
    FOREIGN KEY (publisher_id) REFERENCES Publisher(publisher_id)
);
'''
]

In [7]:
for query in create_table_queries:
    execute_query(connection, query)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


In [8]:
insert_data(connection, "Customer", "customer_id, first_name, last_name, email, phone_number, address", "customer_records.txt")
insert_data(connection, "Book", "book_id, title, isbn, publication_date, price, publisher_id", "book_records.txt")
insert_data(connection, "Author", "author_id, first_name, last_name, birth_date, bio", "author_records.txt")
insert_data(connection, "Publisher", "publisher_id, name, email, phone_number, address", "publisher_records.txt")
insert_data(connection, "`Order`", "order_id, customer_id, order_date, total_amount, shipping_address", "order_records.txt")
insert_data(connection, "BookAuthor", "book_id, author_id", "book_author_records.txt")
insert_data(connection, "OrderBook", "order_id, book_id, quantity", "order_book_records.txt")
insert_data(connection, "AuthorPublisher", "author_id, publisher_id", "author_publisher_records.txt")


Data inserted into Customer
Data inserted into Book
Data inserted into Author
Data inserted into Publisher
Data inserted into `Order`
Data inserted into BookAuthor
Data inserted into OrderBook
Data inserted into AuthorPublisher


In [9]:
connection.close()

In [3]:
%cd {file_path}
!django-admin startproject book_store
%cd book_store

/Users/moeinrazavi/Educational/TAMU_Courses/CSCE 608 - Database Systems/Projects
CommandError: '/Users/moeinrazavi/Educational/TAMU_Courses/CSCE 608 - Database Systems/Projects/book_store' already exists
/Users/moeinrazavi/Educational/TAMU_Courses/CSCE 608 - Database Systems/Projects/book_store


In [4]:
!python manage.py startapp bookstore_app

CommandError: 'bookstore_app' conflicts with the name of an existing Python module and cannot be used as an app name. Please try another name.


In [5]:
!python manage.py makemigrations
!python manage.py migrate

No changes detected
Operations to perform:
  Apply all migrations: admin, auth, bookstore_app, contenttypes, sessions
Running migrations:
  No migrations to apply.


In [6]:
!python manage.py runserver

Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).
March 21, 2023 - 18:07:06
Django version 4.1.7, using settings 'book_store.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.
[21/Mar/2023 18:07:07] "GET / HTTP/1.1" 200 611
[21/Mar/2023 18:07:07] "GET /static/css/styles.css HTTP/1.1" 404 1902
Not Found: /favicon.ico
[21/Mar/2023 18:07:07] "GET /favicon.ico HTTP/1.1" 404 3312
[21/Mar/2023 18:07:10] "GET /list_customers/ HTTP/1.1" 200 1722279
[21/Mar/2023 18:07:10] "GET /static/css/styles.css HTTP/1.1" 404 1902
[21/Mar/2023 18:12:23] "GET /list_customers/ HTTP/1.1" 200 1722279
[21/Mar/2023 18:12:23] "GET /static/css/styles.css HTTP/1.1" 404 1902
[21/Mar/2023 18:12:26] "GET /list_customers/ HTTP/1.1" 200 1722279
[21/Mar/2023 18:12:26] "GET /static/css/styles.css HTTP/1.1" 404 1902
[21/Mar/2023 18:12:49] "GET /list_customers/ HTTP/1.1" 200 1722688
[21/Mar/2023 18:12:54] "GET 

In [ ]:
!python manage.py shell